## 한국경제 뉴스 제목, 작성날짜, url 가져오기

### selenium

In [ ]:
import sys, os
from selenium import webdriver
from selenium.webdriver.common.by import By
from pandas import DataFrame
from bs4 import BeautifulSoup
from datetime import datetime
from tqdm import tqdm

In [ ]:
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

sleep_sec = 0.5

In [ ]:
options = webdriver.ChromeOptions()
browser = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver", options=options)
browser.execute_script('return navigator.userAgent')

In [ ]:
query = input('검색할 키워드  : ')

options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36")
options.add_argument("--blink-settings=imagesEnabled=false")

browser = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver", options=options)

browser.get('https://search.hankyung.com/apps.frm/search.news?query={}&mediaid_clust=HKPAPER,HKCOM&page=1'.format(query))
browser.implicitly_wait(sleep_sec)

news_info = browser.find_element(by=By.XPATH, value='//*[@id="content"]/div[1]/div[2]/div[1]/h3/span[2]').text.split('/')
news_num = news_info[1].replace(" ", "").replace(",", "").replace("건", "")
print(int(news_num))

browser.close()

In [ ]:
news_dict = {}
idx = 0

options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36")
options.add_argument("--blink-settings=imagesEnabled=false")

browser = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver")

for i in tqdm(range(1, int(news_num)//10 + 1)): # 페이지 수
    browser.get('https://search.hankyung.com/apps.frm/search.news?query={}&mediaid_clust=HKPAPER,HKCOM&page={}'.format(query, i))
    browser.implicitly_wait(sleep_sec)

    table = browser.find_element(by=By.XPATH, value='//ul[@class="article"]')
    li_list = table.find_elements(by=By.XPATH, value='./li')

    for li in li_list:
        news_url = li.find_element(by=By.XPATH, value='./div[@class="txt_wrap"]/a').get_attribute('href')
        news_title = li.find_element(by=By.XPATH, value='./div[@class="txt_wrap"]/a/em[@class="tit"]').text
        news_date = li.find_element(by=By.XPATH, value='./div[@class="txt_wrap"]/p[@class="info"]/span[@class="date_time"]').text
        
        news_dict[idx] = {'date' : news_date,
                        'title' : news_title,
                        'url' : news_url,
                        }
        idx += 1
        
browser.close()

In [ ]:
news_df = DataFrame(news_dict).T

folder_path = os.getcwd()
cvs_file_name = '한국경제_{}_{}개_{}.csv'.format(query, news_num, date)

news_df.to_csv(cvs_file_name)

print('데이터 저장 완료 | 경로 : {}\\{}\n'.format(folder_path, cvs_file_name))

news_df.head()

## 한국경제 뉴스 내용 가져오기

### newspaper multiprocessing

In [ ]:
from newspaper import Article
import os
import pandas as pd
from tqdm import tqdm
from multiprocessing.dummy import Pool as ThreadPool

In [ ]:
df = pd.read_excel("/Users/yeonsun/Documents/capstone-2022-29/crawling/한국경제_코로나_93542개_2022-03-28_03시46분.csv")

urls = df["url"].tolist()
news_list = list()

In [ ]:
def getText(url):
    article = Article(url)
    article.download()
    try:
        article.parse()
        news_list.append(
            {
                "title": article.title,
                "content": article.text,
                "date": article.publish_date,
                "time": article.publish_date,
                "url": url,
            }
        )
        return ""
    except:
        news_list.append(
            {
                "title": article.title,
                "content": "",
                "date": article.publish_date,
                "time": article.publish_date,
                "url": url,
            }
        )
        return ""
    
pool = ThreadPool(10)


In [ ]:
# open the urls in their own threads
# and return the results
for _ in tqdm(pool.imap_unordered(getText, urls), total=len(urls)):
    pass

# close the pool and wait for the work to finish
pool.close()
pool.join()

In [ ]:
col_name = ["title", "content", "date", "time", "url"]
news_df = pd.DataFrame(news_list, columns=col_name)

folder_path = os.getcwd()
cvs_file_name = '한국경제_{}_{}개_{}.csv'.format(query, news_num, date)

news_df.to_csv(cvs_file_name)

print('데이터 저장 완료 | 경로 : {}\\{}\n'.format(folder_path, cvs_file_name))

news_df.head()

### newspaper

In [ ]:
from newspaper import Article
import os
import pandas as pd
from tqdm import tqdm

In [ ]:
news_df = pd.read_excel("/Users/yeonsun/Documents/capstone-2022-29/crawling/한국경제_코로나_93542개_2022-03-28_03시46분.csv")
news_df['content'] = ''

contents = []

In [ ]:
def getText(url):
    article = Article(url)
    article.download()
    try:
        article.parse()
        contents.append(article.text)
        return ""
    except:
        contents.append("")
        return ""

In [ ]:
for i in tqdm(range(0, news_df.shape[0])):
    getText(news_df['url'][i])

In [ ]:
idx = 0
for i in range(0, news_df.shape[0]):
    news_df['content'][i] = contents[idx]
    idx += 1

In [ ]:
folder_path = os.getcwd()
cvs_file_name = '한국경제_{}_{}개_{}.csv'.format(query, news_num, date)

news_df.to_csv(cvs_file_name)

print('데이터 저장 완료 | 경로 : {}\\{}\n'.format(folder_path, cvs_file_name))

news_df.head()